# Justin Zarkovacki 2/15/2023
# Transfer Learning K49 -> Kanji

# Prepare imports

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install optuna==3.0.3

# Transfer Learning Notebook

In [3]:
import numpy as np
import os
import optuna
import random

import matplotlib
from matplotlib import pyplot as plt

import tensorflow as tf
from keras.models import Sequential
from keras.layers import Conv2D, Dropout, AveragePooling2D, MaxPooling2D, Flatten, Dense, GlobalAveragePooling2D, Rescaling
from keras import Input, models, backend as K
from tensorflow.keras import layers, models

print("Done!")

Done!


# Function Definitions and Variables

In [4]:
epochs = 12

# def load(f):
#     return np.load(f)['arr_0']

# Helper to create the graphics
def create_visuals(graph_title, model_hist, test_images, test_labels):
    accuracy_data = model_hist.history['accuracy']
    val_accuracy_data = model_hist.history['val_accuracy']

    lower_bound = min(min(accuracy_data), min(val_accuracy_data))

    plt.plot(accuracy_data, label='Train Accuracy')
    plt.plot(val_accuracy_data, label = 'Validation Accuracy')

    plt.title(graph_title)
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.ylim([lower_bound - 0.01, 1])
    plt.legend(loc='lower right')

    print(accuracy_data[-1])
    print(val_accuracy_data[-1])
#     test_loss, test_acc = model_hist.evaluate(test_images, test_labels, verbose=2)

print("Done!")

Done!


In [13]:
im_size = (64, 64)
sample_size = (64, 64, 1)
bs = 128   # batch size

k49_classes = 49
k49_epochs = 15

kanji_classes = 150
kanji_epochs = 15

# Define paths to datasets
k49_train_path = '/content/drive/MyDrive/datasets/k49_train_imgs'
k49_test_path = '/content/drive/MyDrive/datasets/k49_test_imgs'
kanji_path = '/content/drive/MyDrive/datasets/final_dataset'

## Load K49 Data

In [ ]:
# Load K49 Data
# image_dataset_from_directory() loads images using a default interpolation method of bilinear.
# The K49 dataset needs to be bilinearly interpolated up to (64, 64) from (28, 28) 
k49_train = tf.keras.utils.image_dataset_from_directory(k49_train_path, seed=222,
                  color_mode="grayscale", image_size=sample_size, batch_size=bs)

k49_val = tf.keras.utils.image_dataset_from_directory(k49_test_path, seed=222,
                  color_mode="grayscale", image_size=sample_size, batch_size=bs)

print("Done!")

## Load Kanji Data

In [9]:
kanji_train = tf.keras.utils.image_dataset_from_directory(kanji_path, validation_split=0.3, seed=222,
                  subset="training", color_mode="grayscale", image_size=im_size, batch_size=bs)

kanji_val = tf.keras.utils.image_dataset_from_directory(kanji_path, validation_split=0.3, seed=222,
                  subset="validation", color_mode="grayscale", image_size=im_size, batch_size=bs)

print("Done!")

Found 70599 files belonging to 150 classes.
Using 49420 files for training.
Found 70599 files belonging to 150 classes.
Using 21179 files for validation.
Done!


### Rescale Image Data and Training Performance Optimizations

In [ ]:
normalization_layer = tf.keras.layers.Rescaling(1./255)

# Rescale K49 training data
normalized_k49_train = k49_train.map(lambda x, y: (normalization_layer(x), y))
normalized_k49_val = k49_val.map(lambda x, y: (normalization_layer(x), y))

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

# K49 Optimization
k49_train = k49_train.cache().prefetch(buffer_size=AUTOTUNE)
k49_val = k49_val.cache().prefetch(buffer_size=AUTOTUNE)

print("Done!")

Done!


In [11]:
normalization_layer = tf.keras.layers.Rescaling(1./255)

# Rescale Kanji training data
normalized_kanji_train = kanji_train.map(lambda x, y: (normalization_layer(x), y))
normalized_kanji_val = kanji_val.map(lambda x, y: (normalization_layer(x), y))

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [12]:
AUTOTUNE = tf.data.AUTOTUNE

# Kanji Optimization
kanji_train = kanji_train.cache().prefetch(buffer_size=AUTOTUNE)
kanji_val = kanji_val.cache().prefetch(buffer_size=AUTOTUNE)

print("Done!")

Done!


This notebook will create an ensemble model for Kanjij character recognition. It wil be composed of 2 basic models, and one transfer learning model. Knowledge from K49 will be transfered to Kanji.

# Creating Kanji Model 1

In [14]:
def kanji_objective1(trial):
    # Define search space per trial (integer, categorical and floating point values)
    kern_size = trial.suggest_int('kernel_size', 2, 4)
    l1_filters = trial.suggest_int('first_layer_kernel', 32, 64)
    l2_filters = trial.suggest_int('second_layer_kernel', 32, 96)
    l3_filters = trial.suggest_int('third_layer_kernel', 32, 48)
    activations = trial.suggest_categorical('activation', ['relu', 'sigmoid', 'tanh'])
    dropout = trial.suggest_float('dropout', 0.15, 0.3)
    average_pooling_size = trial.suggest_int('average_pooling_size', 2, 4)
    dense_layer_size = trial.suggest_int('dense_layer_size', 64, 128)
    dense_layer_activation = trial.suggest_categorical('dense_layer_activation', ['relu', 'sigmoid', 'tanh'])

    # Design model
    kanji_1_design = Sequential()
    kanji_1_design.add(Conv2D(l1_filters, kernel_size=kern_size, activation=activations, input_shape=sample_size))
    kanji_1_design.add(AveragePooling2D((average_pooling_size, average_pooling_size)))
    kanji_1_design.add(Conv2D(l2_filters, kernel_size=kern_size, activation=activations, input_shape=sample_size))
    kanji_1_design.add(AveragePooling2D((average_pooling_size, average_pooling_size)))
    kanji_1_design.add(Conv2D(l3_filters, kernel_size=kern_size, activation=activations, input_shape=sample_size))
    kanji_1_design.add(Flatten())
    kanji_1_design.add(Dense(dense_layer_size, activation=dense_layer_activation))
    kanji_1_design.add(Dense(kanji_classes))

    kanji_1_design.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=[tf.keras.metrics.SparseCategoricalCrossentropy(), 'accuracy'])

#     print(kanji1_design.summary())

    callback = tf.keras.callbacks.EarlyStopping(monitor='accuracy', patience=3)

    kanji_1_history = kanji_1_design.fit(kanji_train, epochs=kanji_epochs, batch_size=bs,
                    callbacks=callback, validation_data=kanji_val)

    # Important metric for optuna to optimize over
    return kanji_1_history.history['val_accuracy'][-1]

In [ ]:
# Run Study 1
kanji_study1 = optuna.create_study(direction='maximize', study_name="Kanji-1")
kanji_study1.optimize(kanji_objective1, n_trials=10)

[I 2023-03-01 00:29:32,255] A new study created in memory with name: Kanji-1


Epoch 1/15
1545/1545 [==============================] - 6048s 4s/step - loss: 2.4279 - sparse_categorical_crossentropy: 5.6060 - accuracy: 0.4628 - val_loss: 1.2743 - val_sparse_categorical_crossentropy: 3.1706 - val_accuracy: 0.6911
Epoch 2/15
1545/1545 [==============================] - 533s 345ms/step - loss: 0.9724 - sparse_categorical_crossentropy: 2.5976 - accuracy: 0.7600 - val_loss: 0.7754 - val_sparse_categorical_crossentropy: 2.2068 - val_accuracy: 0.8042
Epoch 3/15
1545/1545 [==============================] - 538s 348ms/step - loss: 0.6571 - sparse_categorical_crossentropy: 1.9962 - accuracy: 0.8372 - val_loss: 0.6320 - val_sparse_categorical_crossentropy: 1.9462 - val_accuracy: 0.8383
Epoch 4/15
1545/1545 [==============================] - 545s 353ms/step - loss: 0.5114 - sparse_categorical_crossentropy: 1.7391 - accuracy: 0.8720 - val_loss: 0.5672 - val_sparse_categorical_crossentropy: 1.8315 - val_accuracy: 0.8541
Epoch 5/15
1545/1545 [==============================] - 54

[I 2023-03-01 04:19:32,051] Trial 0 finished with value: 0.8798338174819946 and parameters: {'kernel_size': 4, 'first_layer_kernel': 58, 'second_layer_kernel': 73, 'third_layer_kernel': 36, 'activation': 'sigmoid', 'dropout': 0.21601818983485532, 'average_pooling_size': 2, 'dense_layer_size': 66, 'dense_layer_activation': 'relu'}. Best is trial 0 with value: 0.8798338174819946.


Epoch 1/15
1545/1545 [==============================] - 140s 90ms/step - loss: 4.1817 - sparse_categorical_crossentropy: 7.9716 - accuracy: 0.1217 - val_loss: 3.3543 - val_sparse_categorical_crossentropy: 4.9524 - val_accuracy: 0.2189
Epoch 2/15
1545/1545 [==============================] - 159s 103ms/step - loss: 2.9057 - sparse_categorical_crossentropy: 4.5051 - accuracy: 0.3186 - val_loss: 2.5744 - val_sparse_categorical_crossentropy: 4.3587 - val_accuracy: 0.3769
Epoch 3/15
1545/1545 [==============================] - 156s 101ms/step - loss: 2.3162 - sparse_categorical_crossentropy: 4.1051 - accuracy: 0.4419 - val_loss: 2.1554 - val_sparse_categorical_crossentropy: 4.1408 - val_accuracy: 0.4658
Epoch 4/15
1545/1545 [==============================] - 137s 89ms/step - loss: 1.9309 - sparse_categorical_crossentropy: 3.8275 - accuracy: 0.5269 - val_loss: 1.8269 - val_sparse_categorical_crossentropy: 3.8838 - val_accuracy: 0.5420
Epoch 5/15
1545/1545 [==============================] - 13

[I 2023-03-01 05:01:33,477] Trial 1 finished with value: 0.8323339223861694 and parameters: {'kernel_size': 3, 'first_layer_kernel': 42, 'second_layer_kernel': 87, 'third_layer_kernel': 35, 'activation': 'sigmoid', 'dropout': 0.24671028956587188, 'average_pooling_size': 4, 'dense_layer_size': 86, 'dense_layer_activation': 'sigmoid'}. Best is trial 0 with value: 0.8798338174819946.


Epoch 1/15
1545/1545 [==============================] - 228s 147ms/step - loss: 3.1474 - sparse_categorical_crossentropy: 6.7086 - accuracy: 0.3577 - val_loss: 1.8841 - val_sparse_categorical_crossentropy: 3.3058 - val_accuracy: 0.6081
Epoch 2/15
1545/1545 [==============================] - 209s 136ms/step - loss: 1.3405 - sparse_categorical_crossentropy: 2.5938 - accuracy: 0.7215 - val_loss: 0.9761 - val_sparse_categorical_crossentropy: 2.1919 - val_accuracy: 0.7897
Epoch 3/15
 305/1545 [====>.........................] - ETA: 2:28 - loss: 0.9177 - sparse_categorical_crossentropy: 2.0296 - accuracy: 0.8083

In [ ]:
# Print the info from the best trial
print(f'Best trial info:\n{kanji_study1.best_trial}\n')
for param, value in kanji_study1.best_params.items():
    print(f'Param: {param}\tValue: {value}')

# 0.9122243523597717 and parameters: {'kernel_size': 2, 'first_layer_kernel': 62, 'second_layer_kernel': 69, 'third_layer_kernel': 46, 'activation': 'tanh', 'dropout': 0.1848701184406482, 'average_pooling_size': 3, 'dense_layer_size': 102, 'dense_layer_activation': 'relu'}

In [ ]:
# Optuna doesn't save the best model. You must rebuild it and save it.
kern_size	= 
l1_filters = 
l2_filters = 
l2_filters = 
activations = ""
dropout	= 0.
average_pooling_size = 
dense_layer_size	= 
dense_layer_activation = ""

kanji_1 = Sequential()
kanji_1.add(Conv2D(l1_filters, kernel_size=kern_size, activation=activations, input_shape=sample_shape))
kanji_1.add(AveragePooling2D((average_pooling_size, average_pooling_size)))
kanji_1.add(Conv2D(l2_filters, kernel_size=kern_size, activation=activations, input_shape=sample_shape))
kanji_1.add(AveragePooling2D((average_pooling_size, average_pooling_size)))
kanji_1.add(Conv2D(l3_filters, kernel_size=kern_size, activation=activations, input_shape=sample_shape))
kanji_1.add(Flatten())
kanji_1.add(Dense(dense_layer_size, activation=dense_layer_activation))
kanji_1.add(Dense(kanji_classes))

kanji_1.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=[tf.keras.metrics.SparseCategoricalCrossentropy(), 'accuracy'])

In [ ]:
tf.config.run_functions_eagerly(True)

callback = tf.keras.callbacks.EarlyStopping(monitor='accuracy', patience=3)

kanji_1_optuna_history = kanji_1.fit(kanji_train_images, kanji_train_labels, epochs=k49_epochs, batch_size=batches,
                    callbacks=callback, validation_data=(kanji_test_images, kanji_test_labels))
kanji_1.save('/content/drive/MyDrive/saved_models/kanji_1.h5', save_format='h5')

# Creating Kanji Model 2

In [ ]:
def kanji_objective2(trial):
    # Define search space per trial (integer, categorical and floating point values)
    kern_size = trial.suggest_int('kernel_size', 2, 4)
    l1_filters = trial.suggest_int('first_layer_kernel', 32, 54)
    l2_filters = trial.suggest_int('second_layer_kernel', 20, 64)
    l3_filters = trial.suggest_int('third_layer_kernel', 32, 64)
    activations = trial.suggest_categorical('activation', ['relu', 'sigmoid', 'tanh'])
    dropout = trial.suggest_float('dropout', 0.15, 0.3)
    average_pooling_size = trial.suggest_int('average_pooling_size', 2, 4)
    dense_layer_size = trial.suggest_int('dense_layer_size', 64, 128)
    dense_layer_activation = trial.suggest_categorical('dense_layer_activation', ['relu', 'sigmoid', 'tanh'])

    # Design model
    kanji_2_design = Sequential()
    kanji_2_design.add(Conv2D(l1_filters, kernel_size=kern_size, activation=activations, input_shape=sample_size))
    kanji_2_design.add(Conv2D(l2_filters, kernel_size=kern_size, activation=activations, input_shape=sample_size))
    kanji_2_design.add(AveragePooling2D((average_pooling_size, average_pooling_size)))
    kanji_2_design.add(Dropout(dropout))
    kanji_2_design.add(Conv2D(l3_filters, kernel_size=kern_size, activation=activations, input_shape=sample_size))
    kanji_2_design.add(Flatten())
    kanji_2_design.add(Dense(dense_layer_size, activation=dense_layer_activation))
    kanji_2_design.add(Dense(kanji_classes))

    kanji_2_design.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=[tf.keras.metrics.SparseCategoricalCrossentropy(), 'accuracy'])

#     print(kanji_2_design.summary())

    callback = tf.keras.callbacks.EarlyStopping(monitor='accuracy', patience=3)

    kanji_2_history = kanji_2_design.fit(kanji_train_images, kanji_train_labels, epochs=kanji_epochs, batch_size=batches,
                    callbacks=callback, validation_data=(kanji_test_images, kanji_test_labels))

    # Important metric for optuna to optimize over
    return kanji_2_history.history['val_accuracy'][-1]

In [ ]:
# Run Study 2
kanji_study2 = optuna.create_study(direction='maximize', study_name="Kanji-2")
kanji_study2.optimize(kanji_objective2, n_trials=10)

In [ ]:
# Print the info from the best trial
print(f'Best trial info:\n{kanji_study2.best_trial}\n')
for param, value in kanji_study2.best_params.items():
    print(f'Param: {param}\tValue: {value}')

In [ ]:
# Optuna doesn't save the best model. You must rebuild it and save it.
kern_size	= 
l1_filters = 
l2_filters = 
l2_filters = 
activations = ""
dropout	= 0.
average_pooling_size = 
dense_layer_size	= 
dense_layer_activation = ""

kanji_2 = Sequential()
kanji_2.add(Conv2D(l1_filters, kernel_size=kern_size, activation=activations, input_shape=sample_shape))
kanji_2.add(Conv2D(l2_filters, kernel_size=kern_size, activation=activations, input_shape=sample_shape))
kanji_2.add(AveragePooling2D((average_pooling_size, average_pooling_size)))
kanji_2_design.add(Dropout(dropout))
kanji_2.add(Conv2D(l3_filters, kernel_size=kern_size, activation=activations, input_shape=sample_shape))
kanji_2.add(Flatten())
kanji_2.add(Dense(dense_layer_size, activation=dense_layer_activation))
kanji_2.add(Dense(kanji_classes))

kanji_2.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=[tf.keras.metrics.SparseCategoricalCrossentropy(), 'accuracy'])

In [ ]:
tf.config.run_functions_eagerly(True)

callback = tf.keras.callbacks.EarlyStopping(monitor='accuracy', patience=3)

kanji_2_optuna_history = kanji_2.fit(kanji_train_images, kanji_train_labels, epochs=kanji_epochs, batch_size=batches,
                    callbacks=callback, validation_data=(kanji_test_images, kanji_test_labels))
kanji_2.save('/content/drive/MyDrive/saved_models/kanji_2.h5', save_format='h5')

# KMNIST to K49 Transfer Learning

In [ ]:
import os
import random
from skimage.transform import rescale
from skimage import io

# Loading a base model requires you pop all dimensionality flattening layers
def load_base_model(filename):
    bm = models.load_model(filename)
    bm.pop()
    bm.pop()
    bm.pop()
    bm.trainable = False
    return bm

## Create K49 Base

In [ ]:
def k49_base_objective(trial):
    # Define search space per trial (integer, categorical and floating point values)
    kern_size = trial.suggest_int('kernel_size', 2, 3)
    l1_filters = trial.suggest_int('first_layer_kernel', 20, 40)
    l2_filters = trial.suggest_int('second_layer_kernel', 40, 64)
    activations = trial.suggest_categorical('activation', ['relu', 'sigmoid', 'tanh'])
    dropout = trial.suggest_float('dropout', 0.15, 0.3)
    average_pooling_size = trial.suggest_int('average_pooling_size', 2, 4)
    dense_layer_size = trial.suggest_int('dense_layer_size', 64, 80)
    dense_layer_activation = trial.suggest_categorical('dense_layer_activation', ['relu', 'sigmoid', 'tanh'])

    # Design model
    k49_base_design = Sequential()
    k49_base_design.add(Conv2D(l1_filters, kernel_size=kern_size, activation=activations, input_shape=sample_shape))
    k49_base_design.add(Dropout(dropout))
    k49_base_design.add(Conv2D(l2_filters, kernel_size=kern_size, activation=activations, input_shape=sample_shape))
    k49_base_design.add(AveragePooling2D((average_pooling_size, average_pooling_size)))
    k49_base_design.add(Flatten())
    k49_base_design.add(Dense(dense_layer_size, activation=dense_layer_activation))
    k49_base_design.add(Dense(k49_classes))

    k49_base_design.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=[tf.keras.metrics.SparseCategoricalCrossentropy(), 'accuracy'])

#     print(k49_base_design.summary())

    callback = tf.keras.callbacks.EarlyStopping(monitor='accuracy', patience=3)

    k49_base_history = k49_base_design.fit(k49_train_images, k49_train_labels,
                    epochs=k49_epochs, batch_size=batches, callbacks=callback,
                    validation_data=(k49_test_images, k49_test_labels))

    # Important metric for optuna to optimize over
    return k49_base_history.history['val_accuracy'][-1]

In [ ]:
# Run Study 3
k49_base_study = optuna.create_study(direction='maximize', study_name="K49-Base")
k49_base_study.optimize(kmnist_base_objective, n_trials=15)

In [ ]:
# Print the info from the best trial
print(f'Best trial info:\n{k49_base_study.best_trial}\n')
for param, value in k49_base_study.best_params.items():
    print(f'Param: {param}\tValue: {value}')

In [ ]:
# Optuna doesn't save the best model. You must rebuild it and save it.
kern_size	= 
l1_filters = 
l2_filters = 
activations = ""
dropout	= 0.
average_pooling_size = 
dense_layer_size	= 
dense_layer_activation = ""

k49_base_design = Sequential()
k49_base_design.add(Conv2D(l1_filters, kernel_size=kern_size, activation=activations, input_shape=sample_shape))
k49_base_design.add(Dropout(dropout))
k49_base_design.add(Conv2D(l2_filters, kernel_size=kern_size, activation=activations, input_shape=sample_shape))
k49_base_design.add(AveragePooling2D((average_pooling_size, average_pooling_size)))
k49_base_design.add(Flatten())
k49_base_design.add(Dense(dense_layer_size, activation=dense_layer_activation))
k49_base_design.add(Dense(k49_classes))

kmnist_base.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=[tf.keras.metrics.SparseCategoricalCrossentropy(), 'accuracy'])

In [ ]:
tf.config.run_functions_eagerly(True)

callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

k49_base_optuna_history = k49_base.fit(k49_train_images, k49_train_labels, epochs=k49_epochs, batch_size=batches,
                    callbacks=callback, validation_data=(k49_test_images, k49_test_labels))

k49_base.save('/content/drive/MyDrive/saved_models/k49_base.h5', save_format='h5')

## Create Kanji Top

In [ ]:
def kanji_top_objective(trial):
    # Define search space per trial (integer, categorical and floating point values)
    kern_size = trial.suggest_int('kernel_size', 2, 3)
    l1_filters = trial.suggest_int('first_layer_kernel', 32, 54)
    l2_filters = trial.suggest_int('second_layer_kernel', 64, 96)
    l1_activation = trial.suggest_categorical('first_layer_activation', ['relu', 'sigmoid', 'tanh'])
    l2_activation = trial.suggest_categorical('second_layer_activation', ['relu', 'sigmoid', 'tanh'])
    dropout = trial.suggest_float('dropout', 0.15, 0.3)
    average_pooling_size = trial.suggest_int('average_pooling_size', 2, 4)
    dense_layer_size = trial.suggest_int('dense_layer_size', 64, 128)
    dense_layer_activation = trial.suggest_categorical('dense_layer_activation', ['relu', 'sigmoid', 'tanh'])

    base_model = load_base_model("/content/drive/MyDrive/saved_models/k49_base.h5")
    
    # Design model
    kanji_top_design = Sequential()
    kanji_top_design.add(base_model)
    kanji_top_design.add(Conv2D(l1_filters, kernel_size=kern_size, activation=l1_activation, input_shape=k49_input_shape))
    kanji_top_design.add(Dropout(dropout))
    kanji_top_design.add(Conv2D(l2_filters, kernel_size=kern_size, activation=l2_activation, input_shape=k49_input_shape))
    kanji_top_design.add(AveragePooling2D((average_pooling_size, average_pooling_size)))
    kanji_top_design.add(Flatten())
    kanji_top_design.add(Dense(dense_layer_size, activation=dense_layer_activation))
    kanji_top_design.add(Dense(k49_classes))

    kanji_top_design.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=[tf.keras.metrics.SparseCategoricalCrossentropy(), 'accuracy'])

#     print(kanji_top_design.summary())

    callback = tf.keras.callbacks.EarlyStopping(monitor='accuracy', patience=3)

    kanji_top_history = kanji_top_design.fit(??, ??, epochs=k49_epochs, batch_size=batches,
                    callbacks=callback, validation_data=(??, ??))

    # Important metric for optuna to optimize over
    return kanji_top_history.history['val_accuracy'][-1]

In [ ]:
# Figure out if this is a good design ^